# Prim algorithm with heap data structure 

In [86]:
import math
import time
from collections import defaultdict

## MinHeap data structure implementation

In [87]:
# ArrayHeap object extends list
class ArrayHeap(list):
    def __init__(self, array):
        super().__init__(array)
        self.heapSize = len(array)

class MinHeap:
    def __init__(self, array):
        self.arrayHeap = ArrayHeap(array)
    
    # All the following methods work with zero based array. Hence, we need to handle separately odd and even indexes.
    def parent(self, i):
        if i%2 == 0: # even
            return math.floor(i/2) - 1
        else:
            return math.floor(i/2)
    
    def left(self, i):
        return 2*i + 1
    
    def right(self, i):
        return 2*i + 2
    
    # Execution time: O(lg n)
    def minHeapify(self, i):
        l = self.left(i)
        r = self.right(i)
        if l <= self.arrayHeap.heapSize-1 and self.arrayHeap[l].key < self.arrayHeap[i].key:
            minimo = l
        else:
            minimo = i
        if r <= self.arrayHeap.heapSize-1 and self.arrayHeap[r].key < self.arrayHeap[minimo].key:
            minimo = r
        if minimo != i:
            self.arrayHeap[i], self.arrayHeap[minimo] = self.arrayHeap[minimo], self.arrayHeap[i]
            self.minHeapify(minimo)
            
    # Execution time: O(n)
    def buildMinHeap(self):
        for i in range(math.floor(len(self.arrayHeap)/2), -1, -1): #downto
            self.minHeapify(i)

    def bubbleUp(self, index):
        parent = self.parent(index)
        current = index
        while current > 0 and self.arrayHeap[parent].key > self.arrayHeap[current].key:
            self.arrayHeap[current], self.arrayHeap[parent] = self.arrayHeap[parent], self.arrayHeap[current]
            current = parent
            parent = self.parent(parent)

    # Execution time: O(lg n)
    # First we update the heap structure, then we remove the last element.
    def extractMin(self):
        if self.arrayHeap.heapSize < 0:
            print("Error: extractMin underflow")
            return
        else:
            minimo = self.arrayHeap[0] # Save the minimum node
            self.arrayHeap[0].isPresent = False # Set its flag to false
            self.arrayHeap[0] = self.arrayHeap[self.arrayHeap.heapSize - 1] # Assign the first node the last one
            self.arrayHeap.pop(self.arrayHeap.heapSize-1) # Pop the last node
            self.arrayHeap.heapSize -= 1 # Decreasing heapsize
            self.minHeapify(0) # Call minHeapify in order to move the new first node to the correct position

            return minimo

## Definition of Node and Graph objects for graph manipluation        

In [88]:
class Node:
    def __init__(self, tag):
        self.tag = tag
        self.key = None
        self.parent = None
        self.isPresent = True
        self.adjacencyList = []
        
    # For test 
    def print(self):
        print("tag =", self.tag, "adjList=", self.adjacencyList, "key=", self.key)

class Graph:
    def __init__(self):
        self.nodes = defaultdict(Node)
        
    def createNodes(self, nums):
        for i in range(1, nums+1): # nums+1 in order to cover the last node
            self.nodes[i] = Node(i)

    def addNode(self, tag, adjList):
        adjTag = adjList[0]
        adjCost = adjList[1]
        self.nodes[tag].adjacencyList.append([self.nodes[adjTag], adjCost])
        self.nodes[adjTag].adjacencyList.append([self.nodes[tag], adjCost]) # Graph is undirected

    def buildGraph(self, input):
        lines = input.readlines()
        self.createNodes(int(lines[0].split()[0])) # Extract number of vertexes and pass it to createNode
        lines.pop(0) # Remove the first line of .txt input file
        for line in lines:
            info = list(map(int, line.split())) # Convert all the strings deriving from split to int
            self.addNode(info[0], [info[1], info[2]])

## Prim algorithm

In [89]:
def MSTPrim(g, r):
    for node in g.nodes.values():
        node.key = math.inf # Set key. Parent is already set through Node constructor.
    r.key = 0
    q = MinHeap(list(g.nodes.values()))
    q.buildMinHeap() # This function call is necessary in order to call MSTPrim with any root node
    while len(q.arrayHeap) is not 0:
        u = q.extractMin()
        for v in u.adjacencyList:
            if v[0].isPresent and v[1] < v[0].key:
                v[0].parent = u
                v[0].key = v[1]
                q.bubbleUp(q.arrayHeap.index(v[0])) # bubbleUp maintains the minheap condition

## Main

In [90]:
inp = open("dataset/input_random_01_10.txt", "r")
lines = inp.readlines()
start = time.time()
result = Graph()
result.buildGraph(open("dataset/input_random_01_10.txt", "r"))

startingNode = 1 # Root node value
MSTPrim(result, result.nodes.get(startingNode))
print("partenza = nodo", startingNode)
print("Execution time =", time.time() - start)
sum = 0
for node in result.nodes.values():
    sum += node.key
print(sum, "\n")


partenza = nodo 1
Execution time = 12.774907350540161
